**Connect to the Oscilloscope**

In [ ]:
import visa

# Creates instance of the ActiveDSO control
rm = visa.ResourceManager()

# Connects to the oscilloscope.
lecroy = rm.open_resource("TCPIP0::<hostname|IP address>::INSTR")

# Set up 'time out' error to appear after 5 sec
lecroy.timeout = 5000

# Clear the buffers sent to the oscilloscope before
lecroy.clear() 

**Set up the "environment"**

In [ ]:
# Tells the oscilloscope  "don't return the commandheader with query result."
lecroy.write("COMM_HEADER OFF")

# Recall the default setup
lecroy.write(r"""vbs 'app.settodefaultsetup' """)

# Set up the wait time of receiving new acquisition
lecroy.query(r"""vbs? 'return=app.WaitUntilIdle(5)' """)

# Stop the previous acquisition first
lecroy.write(r"""vbs 'app.acquisition.triggermode = "stopped" ' """)

# Set up trigger
lecroy.write(r"""vbs 'app.acquisition.trigger.edge.level = <value>' """)
lecroy.write(r"""vbs 'app.acquisition.triggermode = "single" ' """)

# Set up horizontal maximize value
lecroy.write(r"""vbs 'app.acquisition.horizontal.maximize = "<value>" ' """)

# Clear previsous setup measurements
lecroy.write(r"""vbs 'app.measure.clearall ' """)
lecroy.write(r"""vbs 'app.measure.clearsweeps ' """)

# Set up new measurements
# Turn on measurements, statistics, parameter
lecroy.write(r"""vbs 'app.measure.showmeasure = true ' """)
lecroy.write(r"""vbs 'app.measure.statson = true ' """)
lecroy.write(r"""vbs 'app.measure.p1.view = true ' """)
# Set the parameter we want to measure
lecroy.write(r"""vbs 'app.measure.p1.paramengine = "<value>" ' """)
# Set the parameter source
lecroy.write(r"""vbs 'app.measure.p1.source1 = "C1" ' """)


**Set up control variables**

In [ ]:
# app.Acquisition has a lot of command to change the variables
# i.e. Set the C1 V/div to 20mV
lecroy.write(r"""vbs 'app.Acquisition.C1.VDIV=".02 V" ' """, 1)

**Acquire**

In [ ]:
# Set up acquisitions numbers: epochs
for i in range(0,epochs):
    # Forcing a trigger if none occurs after 0.1seconds
    r = lecroy.query(r"""vbs? 'return=app.acquisition.acquire( 0.1 , True )' """)
    # Wait for 5 secs after all aquisitions is completed before measuering
    r = lecroy.query(r"""vbs? 'return=app.WaitUntilIdle(5)' """)
    if r==0:
        print("Time out from WaitUntilIdle, return = {0}".format(r))

**Read the result**

In [ ]:
<var> = lecroy.query(r"""vbs? 'return=app.measure.p1.out.result.value' """)
print("<var> = {0}".format(<var>))

**Disconnect**

In [ ]:
lecroy.close()
rm.close()